In [ ]:
!pip3 install torch torchvision

In [ ]:
!pip install easyocr

In [ ]:
import easyocr
import cv2
import numpy as np
from matplotlib import pyplot as plt
import torch
import json


In [ ]:
IMAGE_PATH = 'Images\i1.png'

In [ ]:
# Process image and extract text
reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
result = reader.readtext(IMAGE_PATH,width_ths=0.7)  # width_ths=0.7 for better grouping of characters without this the field inputs get split into multiple slices
print(result)

In [ ]:
def get_y_coordinate(item):
    return item[0][0][1]

In [ ]:
# Sort all results by their y-coordinate first
sorted_results = sorted(result, key=get_y_coordinate)

# Group text into lines by first checking the y-coordinate proximity and then sorting by x-coordinate within each line
lines = []
if sorted_results:
    current_line = [sorted_results[0]]
    for i in range(1, len(sorted_results)):
        # Define a vertical threshold = half of the height of the first text in the line
        # Use the y-coordinate of the first item in the current line as a reference
        if abs(get_y_coordinate(sorted_results[i]) - get_y_coordinate(current_line[0])) < (current_line[0][0][3][1]-current_line[0][0][0][1])/2:
            current_line.append(sorted_results[i])
        else:
            lines.append(sorted(current_line, key=lambda x: x[0][0][0]))  # Sort current line by x-coordinate
            current_line = [sorted_results[i]]
    lines.append(sorted(current_line, key=lambda x: x[0][0][0])) # Sort and add the last line

In [ ]:
#view output for debugging. REMOVE IN FINAL CODE
print("\nExtracted Text by Lines:")
for line in lines:
    line_text = ':'.join([item[1] for item in line])
    print(line_text)

In [ ]:
# crude json output extraction
extracted_data = {}
for line in lines:          
    if len(line) >= 2:
        label = line[0][1]
        value = line[1][1]
        extracted_data[label] = value # detected labels should be unique 

In [ ]:
output_json_path = 'extracted_data.json'

# Write the dictionary to a JSON file
with open(output_json_path, 'w') as json_file:
    json.dump(extracted_data, json_file, indent=4)

print(f"Data successfully saved to {output_json_path}")